# 垃圾分拣玩法  Garbage sorting play

### 导入头文件 Import head file

In [1]:
#!/usr/bin/env python
# coding: utf-8
import Arm_Lib
import os
import cv2 as cv
import threading
import time
import ipywidgets as widgets
from IPython.display import display
from garbage_identify import garbage_identify
from dofbot_utils.fps import FPS
from dofbot_utils.dofbot_config import *

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /home/jetson/ultralytics/ultralytics/data/yahboom_data/best.engine for TensorRT inference...
[04/18/2025-18:00:59] [TRT] [I] Loaded engine size: 11 MiB
[04/18/2025-18:00:59] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[04/18/2025-18:01:00] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +19, now: CPU 0, GPU 28 (MiB)


### 初始化机械臂位置 Initialize DOFBOT position

In [2]:
target = garbage_identify()
calibration = Arm_Calibration()
arm = Arm_Lib.Arm_Device()
num=0
dp = []
msg = {}
xy = [90, 105]
debug_pos = False
threshold = 118
model = "General"
# XYT参数路径  XYT Parameter path
XYT_path="/home/jetson/dofbot_pro/dofbot_garbage_yolov11/XYT_config.txt"
try: xy, threshold = read_XYT(XYT_path)
except Exception: print("Read XYT_config Error !!!")

### 创建实例,初始化参数 Create the instance and initialize the parameters

In [3]:
import Arm_Lib
arm = Arm_Lib.Arm_Device()
joints_0 = [xy[0], xy[1], 0, 0, 90, 30]
arm.Arm_serial_servo_write6_array(joints_0, 1000)

### 创建控件 Creating widget

In [4]:
button_layout      = widgets.Layout(width='320px', height='60px', align_self='center')
output = widgets.Output()
# 调整滑杆 Adjust the slider
joint1_slider      = widgets.IntSlider(description='joint1 :'   ,    value=xy[0]     , min=70 , max=110, step=1, orientation='horizontal')
joint2_slider      = widgets.IntSlider(description='joint2 :'   ,    value=xy[1]     , min=90, max=150, step=1, orientation='horizontal')
threshold_slider   = widgets.IntSlider(description='threshold :',    value=threshold , min=0  , max=255, step=1, orientation='horizontal')

# 进入标定模式  Enter calibration mode
calibration_model  = widgets.Button(description='calibration_model',  button_style='primary', layout=button_layout)
calibration_ok     = widgets.Button(description='calibration_ok',     button_style='success', layout=button_layout)
calibration_cancel = widgets.Button(description='calibration_cancel', button_style='danger', layout=button_layout)

# 目标检测抓取
target_detection   = widgets.Button(description='target_detection', button_style='info', layout=button_layout)
grap = widgets.Button(description='grap', button_style='success', layout=button_layout)
# 退出  exit
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
garbage_identify = widgets.VBox(
    [joint1_slider, joint2_slider, threshold_slider, calibration_model, calibration_ok, calibration_cancel, target_detection, grap,exit_button],
    layout=widgets.Layout(align_self='center'));
controls_box = widgets.HBox([imgbox, garbage_identify], layout=widgets.Layout(align_self='center'))

### 标定回调 Calibration callback function

In [5]:
def calibration_model_Callback(value):
    global model
    model = 'Calibration'
    with output: print(model)
def calibration_OK_Callback(value):
    global model
    model = 'calibration_OK'
    with output: print(model)
def calibration_cancel_Callback(value):
    global model
    model = 'calibration_Cancel'
    with output: print(model)
calibration_model.on_click(calibration_model_Callback)
calibration_ok.on_click(calibration_OK_Callback)
calibration_cancel.on_click(calibration_cancel_Callback)

### 模式切换   switching mode

In [6]:
def target_detection_Callback(value):
    global model,debug_pos
    model = 'Detection'
    with output: print(model)
    debug_pos = True
def grap_Callback(value):
    global model
    model = 'Grap'
    with output: print(model)
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output: print(model)
target_detection.on_click(target_detection_Callback)
grap.on_click(grap_Callback)
exit_button.on_click(exit_button_Callback)

### 主程序 Main process

In [7]:
def camera():
    global model,dp,msg,debug_pos
    # 打开摄像头 Open camera
    capture = cv.VideoCapture(0, cv.CAP_V4L2)
    capture.set(cv.CAP_PROP_FRAME_WIDTH, 640)
    capture.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
    m_fps = 0
    t_start = time.time()
    fps = FPS()
    # Be executed in loop when the camera is opened normally 
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            _, img = capture.read()
            xy=[joint1_slider.value,joint2_slider.value]
            if model == 'Calibration':
                _, img = calibration.calibration_map(img,xy,threshold_slider.value)
            if model == 'calibration_OK':
                try: write_XYT(XYT_path,xy, threshold_slider.value)
                except Exception: print("File XYT_config Error !!!")
                dp, img = calibration.calibration_map(img,xy,threshold_slider.value)
                model="General"
            if len(dp) != 0: img = calibration.Perspective_transform(dp, img)
            if model == 'calibration_Cancel':  
                dp = []
                msg= {}
                model="General"
            if len(dp)!= 0 and  model == 'Detection': 
                img, msg = target.garbage_run(img)
                if debug_pos:
                    debug_pos = False
                    print("detect msg", msg)
            if len(msg)!= 0 and model == 'Grap':
                print("grap msg", msg)
                threading.Thread(target=target.garbage_grap, args=(msg, xy,)).start()
                msg={}
                model="Detection"
            if model == 'Exit':
                capture.release()
                break
            fps.update_fps()
            fps.show_fps(img)

            imgbox.value = cv.imencode('.jpg', img)[1].tobytes()
        except KeyboardInterrupt:
            capture.release()
        except:
            capture.release()

### 启动  Start

In [8]:
display(controls_box,output)
threading.Thread(target=camera, ).start()

Output()

detect msg {'Disposable_chopsticks': (0.03475, 0.1558, -6.28725266987625), 'Newspaper': (-0.0335, 0.20993, 4.533656085479297), 'Fish_bone': (-0.0425, 0.15447, 7.69226979778372)}
grap msg {'Disposable_chopsticks': (0.0345, 0.1558, -6.243444014536331), 'Newspaper': (-0.0335, 0.20993, 4.533656085479297), 'Fish_bone': (-0.0425, 0.15447, 7.69226979778372), 'Expired_cosmetics': (0.0325, 0.21233, -4.35149967059425)}
request: 0.0345 0.1558
joints: [77.29857584184013, 23.2821714310909, 89.99999983578061, -21.510218167698497, 83.75655598546366]
request: -0.0335 0.20993
joints: [98.90478826260197, 19.700086835143537, 89.99999962404294, -0.8365034421509421, 94.5336560854793]
